In [1]:
import tensorflow as tf

In [2]:
# Load the diabetes dataset

from sklearn.datasets import load_diabetes

diabetes_dataset = load_diabetes()

In [3]:
# Save the input and target variables

from sklearn.model_selection import train_test_split

data = diabetes_dataset['data']
targets = diabetes_dataset['target']

In [4]:
# Split the data set into training and test sets

train_data, test_data, train_targets, test_targets = train_test_split(data, targets, test_size=0.1)

In [5]:
# Build the model

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = tf.keras.Sequential([
    Dense(128, activation='relu', input_shape=(train_data.shape[1],)),
    Dense(64,activation='relu'),
    tf.keras.layers.BatchNormalization(),
    Dense(64, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1)        
])

In [6]:
# Compile the model
    
model.compile(loss='mse', optimizer="adam", metrics=['mae'])

### Defining a Custom Callback

We define our custom callback using the logs dictionary to access the loss and metric values.
The logs dictionary stores the loss value, along with all of the metrics we are using at the end of a batch or epoch.

We can incorporate information from the logs dictionary into our own custom callbacks.

In [7]:
class LossAndMetricCallback(tf.keras.callbacks.Callback):
    
    # Print the loss after every second batch in the training set
    def on_train_batch_end(self, batch, logs=None):
        if batch%2 == 0:
            print('\n After batch {}, the loss is {:7.2f}'.format(batch, logs['loss']))
    
    # Print the loss after each batch in the test set
    def on_test_batch_end(self, batch, logs=None):
        print('\n After batch {}, the loss is {:7.2f}'.format(batch, logs['loss']))
    
    # Print the loss and mae after each epoch in the training set
    def on_epoch_end(self, epoch, logs=None):
        print('\n Epoch {}: Average Loss is {:7.2f}, mean absolute error is {:7.2f}'.format(epoch, logs['loss']
                                                                                            , logs['mae']))
    # Notify the user when prediction has finished after each batch
    def on_predict_batch_end(self, batch, logs=None):
        print('Finished prediction on batch {}'.format(batch))

    

In [8]:
# Train the model

history = model.fit(train_data, train_targets, epochs=20,
                    batch_size=32, callbacks=[LossAndMetricCallback()], verbose=False)


 After batch 0, the loss is 40133.84

 After batch 2, the loss is 30823.12

 After batch 4, the loss is 34196.52

 After batch 6, the loss is 31098.02

 After batch 8, the loss is 31281.37

 After batch 10, the loss is 25813.80

 After batch 12, the loss is 25389.82

 Epoch 0: Average Loss is 29681.41, mean absolute error is  153.87

 After batch 0, the loss is 34105.68

 After batch 2, the loss is 27500.86

 After batch 4, the loss is 25390.81

 After batch 6, the loss is 32050.20

 After batch 8, the loss is 25587.55

 After batch 10, the loss is 22701.82

 After batch 12, the loss is 33020.29

 Epoch 1: Average Loss is 28615.63, mean absolute error is  150.74

 After batch 0, the loss is 31808.57

 After batch 2, the loss is 21785.64

 After batch 4, the loss is 18827.32

 After batch 6, the loss is 26694.63

 After batch 8, the loss is 20967.12

 After batch 10, the loss is 22416.49

 After batch 12, the loss is 35211.75

 Epoch 2: Average Loss is 25598.78, mean absolute error is 

In [9]:
# Evaluate the model

model_eval = model.evaluate(test_data, test_targets, batch_size=10, 
                            callbacks=[LossAndMetricCallback()], verbose=False)


 After batch 0, the loss is 11797.54

 After batch 1, the loss is 12818.25

 After batch 2, the loss is 10156.40

 After batch 3, the loss is 2374.30

 After batch 4, the loss is 1558.41


In [10]:
# Get predictions from the model

model_pred = model.predict(test_data, batch_size=10,
                           callbacks=[LossAndMetricCallback()], verbose=False)

Finished prediction on batch 0
Finished prediction on batch 1
Finished prediction on batch 2
Finished prediction on batch 3
Finished prediction on batch 4


## Learning Rate Scheduler:

We are going to define a callback to change the learning rate of the optimiser of a model during training. We will do this by specifying the epochs and new learning rates where we would like it to be changed.

First we define the auxillary function that returns the learning rate for each epoch based on our schedule.

In [15]:
# Define the learning rate schedule. The tuples below are (start_epoch, new_lr)


lr_schedule = [
    (4, 0.03), (7, 0.02), (11, 0.005), (15, 0.007)
]

def get_new_epoch_lr(epoch, lr):
    # Checks to see if the new input epoch is listed in the learning rate schedule and if so, 
    # returns learning rate lr_schedule
    epoch_in_sched = [i for i in range(len(lr_schedule)) if lr_schedule[i][0] == int(epoch)]
    
    if len(epoch_in_sched)>0:
        # if the epoch exists in the lr_schedule, we return the corresponding learning rate
        return lr_schedule[epoch_in_sched[0]][1]
    else:
        # return the existing learning rate
        return lr


In [11]:
# Defining the custom callback

class LRScheduler(tf.keras.callbacks.Callback):
    
    def __init__(self, new_lr):
        super(LRScheduler, self).__init__()
        # Add the new learning rate function to our callback
        self.new_lr = new_lr
    
    def on_epoch_begin(self, epoch, logs=None):
        # Make sure that the optimizer we have chosen has a learning rate, raise an error if not
        if not hasattr(self.model.optimizer, 'lr'):
            raise ValueError('Error: Optimizer does not have a learning rate')
        
        # Get the current learning rate
        curr_rate = float(tf.keras.backend.get_value(self.model.optimizer.lr))
        
        # Call the auxilliary function to get the scheduled learning rate for the current epoch
        scheduled_rate = self.new_lr(epoch, curr_rate)
        
        # Set the learning rate to the new scheduled learning rate
        tf.keras.backend.set_value(self.model.optimizer.lr, scheduled_rate)
        print('Learning rate for epoch {} is {:7.3f}'.format(epoch, scheduled_rate))
        

In [12]:
# Build the same model as before

new_model = tf.keras.Sequential([
    Dense(128, activation='relu', input_shape=(train_data.shape[1],)),
    Dense(64,activation='relu'),
    tf.keras.layers.BatchNormalization(),
    Dense(64, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1)        
])

In [13]:
# Compile the model

new_model.compile(loss='mse',
                optimizer="adam",
                metrics=['mae', 'mse'])

In [16]:
# Fit the model with our learning rate scheduler callback

new_history = new_model.fit(train_data, train_targets, epochs=20,
                            batch_size=100, callbacks=[LRScheduler(get_new_epoch_lr)], verbose=False)

Learning rate for epoch 0 is   0.001
Learning rate for epoch 1 is   0.001
Learning rate for epoch 2 is   0.001
Learning rate for epoch 3 is   0.001
Learning rate for epoch 4 is   0.030
Learning rate for epoch 5 is   0.030
Learning rate for epoch 6 is   0.030
Learning rate for epoch 7 is   0.020
Learning rate for epoch 8 is   0.020
Learning rate for epoch 9 is   0.020
Learning rate for epoch 10 is   0.020
Learning rate for epoch 11 is   0.005
Learning rate for epoch 12 is   0.005
Learning rate for epoch 13 is   0.005
Learning rate for epoch 14 is   0.005
Learning rate for epoch 15 is   0.007
Learning rate for epoch 16 is   0.007
Learning rate for epoch 17 is   0.007
Learning rate for epoch 18 is   0.007
Learning rate for epoch 19 is   0.007


### Built-in Learning Rate Scheduler:

**Usage:** `tf.keras.callbacks.LearningRateScheduler(schedule, verbose=0)`

As in our custom callback, the `LearningRateScheduler` in Keras takes a function `schedule` as an argument. 

This function `schedule` should take two arguments:
* The current epoch (as an integer), and
* The current learning rate,

and return new learning rate for that epoch. 

The `LearningRateScheduler` also has an optional `verbose` argument, which prints information about the learning rate if it is set to 1.


In [17]:
# Defining the learning rate schedule function:

def new_lr(epoch, lr):
    if epoch%2 == 0:
        return lr
    else:
        return lr + epoch/1000

In [19]:
# Train the model

history1 = model.fit(train_data, train_targets, epochs=10,
                    callbacks=[tf.keras.callbacks.LearningRateScheduler(new_lr, verbose=1)], verbose=False)


Epoch 00001: LearningRateScheduler reducing learning rate to 0.0010000000474974513.

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0020000000474974513.

Epoch 00003: LearningRateScheduler reducing learning rate to 0.0020000000949949026.

Epoch 00004: LearningRateScheduler reducing learning rate to 0.005000000094994903.

Epoch 00005: LearningRateScheduler reducing learning rate to 0.004999999888241291.

Epoch 00006: LearningRateScheduler reducing learning rate to 0.009999999888241292.

Epoch 00007: LearningRateScheduler reducing learning rate to 0.009999999776482582.

Epoch 00008: LearningRateScheduler reducing learning rate to 0.01699999977648258.

Epoch 00009: LearningRateScheduler reducing learning rate to 0.016999999061226845.

Epoch 00010: LearningRateScheduler reducing learning rate to 0.025999999061226846.


In [20]:
# We can also use lambda functions to schedule lr

history2 = model.fit(train_data, train_targets, epochs=10,
                    callbacks=[tf.keras.callbacks.LearningRateScheduler(lambda x:1/(3+5*x), verbose=1)], 
                    verbose=False)


Epoch 00001: LearningRateScheduler reducing learning rate to 0.3333333333333333.

Epoch 00002: LearningRateScheduler reducing learning rate to 0.125.

Epoch 00003: LearningRateScheduler reducing learning rate to 0.07692307692307693.

Epoch 00004: LearningRateScheduler reducing learning rate to 0.05555555555555555.

Epoch 00005: LearningRateScheduler reducing learning rate to 0.043478260869565216.

Epoch 00006: LearningRateScheduler reducing learning rate to 0.03571428571428571.

Epoch 00007: LearningRateScheduler reducing learning rate to 0.030303030303030304.

Epoch 00008: LearningRateScheduler reducing learning rate to 0.02631578947368421.

Epoch 00009: LearningRateScheduler reducing learning rate to 0.023255813953488372.

Epoch 00010: LearningRateScheduler reducing learning rate to 0.020833333333333332.


## CSV logger

**Usage** `tf.keras.callbacks.CSVLogger(filename, separator=',', append=False)`

This callback streams the results from each epoch into a CSV file.
The first line of the CSV file will be the names of pieces of information recorded on each subsequent line, beginning with the epoch and loss value. The values of metrics at the end of each epoch will also be recorded.

The only compulsory argument is the `filename` for the log to be streamed to. This could also be a filepath.

You can also specify the `separator` to be used between entries on each line.

The `append` argument allows you the option to append your results to an existing file with the same name. This can be particularly useful if you are continuing training.


In [22]:
# Train the model with a CSV logger

history3 = model.fit(train_data, train_targets, epochs=10,
                    callbacks=[tf.keras.callbacks.CSVLogger("results.csv")], verbose=False)

In [23]:
# Load the CSV

import pandas as pd

pd.read_csv("results.csv", index_col='epoch')

,loss,mae
epoch,,
0,24999.678443,137.74657
1,24903.073663,137.40007
2,24808.604750,137.05772
3,24716.943138,136.71483
4,24623.719016,136.37712
5,24532.615928,136.04553
6,24443.676239,135.71327
7,24355.196073,135.38320
8,24266.725943,135.06126


## Lambda callbacks

**Usage** `tf.keras.callbacks.LambdaCallback(
        on_epoch_begin=None, on_epoch_end=None, 
        on_batch_begin=None, on_batch_end=None, 
        on_train_begin=None, on_train_end=None)`

Lambda callbacks are used to quickly define simple custom callbacks with the use of lambda functions.

Each of the functions require some positional arguments.
* `on_epoch_begin` and `on_epoch_end` expect two arguments: `epoch` and `logs`,
* `on_batch_begin` and `on_batch_end` expect two arguments: `batch` and `logs` and
* `on_train_begin` and `on_train_end` expect one argument: `logs`.

Let's see an example of this in practice.

In [25]:
# Print the epoch number at the beginning of each epoch

epoch_callback = tf.keras.callbacks.LambdaCallback(
on_epoch_begin=lambda epoch, logs: print('Starting epoch {}'.format(epoch+1)))


In [26]:
# Print the loss at the end of each batch

batch_loss_callback = tf.keras.callbacks.LambdaCallback(
    on_batch_end=lambda batch,logs: print('\n After batch {}, the loss is {:7.2f}.'.format(batch, logs['loss'])))

In [27]:
# Inform that training is finished

train_finish_callback = tf.keras.callbacks.LambdaCallback(
    on_train_end=lambda logs: print('Training finished!'))

In [28]:
# Train the model with the lambda callbacks

history4 = model.fit(train_data, train_targets, epochs=5, batch_size=100,
                    callbacks=[epoch_callback, batch_loss_callback,train_finish_callback], verbose=False)

Starting epoch 1

 After batch 0, the loss is 22544.61.

 After batch 1, the loss is 26039.82.

 After batch 2, the loss is 23047.82.

 After batch 3, the loss is 24878.44.
Starting epoch 2

 After batch 0, the loss is 26565.73.

 After batch 1, the loss is 24514.89.

 After batch 2, the loss is 22227.91.

 After batch 3, the loss is 23044.15.
Starting epoch 3

 After batch 0, the loss is 22309.18.

 After batch 1, the loss is 24282.60.

 After batch 2, the loss is 24771.15.

 After batch 3, the loss is 24940.61.
Starting epoch 4

 After batch 0, the loss is 24256.11.

 After batch 1, the loss is 21205.49.

 After batch 2, the loss is 24652.61.

 After batch 3, the loss is 26120.96.
Starting epoch 5

 After batch 0, the loss is 25912.91.

 After batch 1, the loss is 19946.36.

 After batch 2, the loss is 24268.54.

 After batch 3, the loss is 26000.64.
Training finished!


## Reduce learning rate on plateau

**Usage** `tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss', 
            factor=0.1, 
            patience=10, 
            verbose=0, 
            mode='auto', 
            min_delta=0.0001, 
            cooldown=0, 
            min_lr=0)`

The `ReduceLROnPlateau` callback allows reduction of the learning rate when a metric has stopped improving. 
The arguments are similar to those used in the `EarlyStopping` callback.
* The argument `monitor` is used to specify which metric to base the callback on.
* The `factor` is the factor by which the learning rate decreases i.e., new_lr=factor*old_lr.
* The `patience` is the number of epochs where there is no improvement on the monitored metric before the learning rate is reduced.
* The `verbose` argument will produce progress messages when set to 1.
* The `mode` determines whether the learning rate will decrease when the monitored quantity stops increasing (`max`) or decreasing (`min`). The `auto` setting causes the callback to infer the mode from the monitored quantity.
* The `min_delta` is the smallest change in the monitored quantity to be deemed an improvement.
* The `cooldown` is the number of epochs to wait after the learning rate is changed before the callback resumes normal operation.
* The `min_lr` is a lower bound on the learning rate that the callback will produce.


In [29]:
# Train the model with the ReduceLROnPlateau callback

history5 = model.fit(train_data, train_targets, epochs=100, batch_size=100,
                    callbacks=[tf.keras.callbacks.ReduceLROnPlateau(
                        monitor="loss",factor=0.2, verbose=1)], verbose=False)